Natural Convection
===

Coupling of Navier-Stokes and heat transport

* Change in temperature leads to gravity forces
* Temperature is convected by fluid velocity

Rayleigh-Benard benchmark example

In [1]:
from ngsolve import *
import netgen.geom2d
import netgen.meshing
from ngsolve.webgui import Draw
import math

from ngs_templates.NavierStokes import *
from ngs_templates.ConvectionDiffusion import * 

ngsglobals.msg_level = 0

In [2]:
geo = netgen.geom2d.SplineGeometry()
geo.AddRectangle( (0,0), (0.06, 0.01), bcs=['b','r','t','l'])
mesh = Mesh(geo.GenerateMesh(maxh=0.002))

In [3]:
timestep = 0.1
navstokes = NavierStokes (mesh, nu=1.04177e-6, order=3, timestep = timestep,
                              inflow="", outflow="", wall="l|r|b|t", uin=(0,0) )

Tinitial = 293.5-50*y+y*(0.01-y)*1e3*sin(20/0.06*x*math.pi)

convdiff = ConvectionDiffusionEquation (mesh, order=3, lam=1.38e-7, \
                wind = navstokes.velocity, dirichlet="b|t", udir=Tinitial, timestep=timestep)
convdiff.SetInitial(Tinitial)

T0 = 293
beta = 2.07e-4
navstokes.AddForce ( (1-beta*(convdiff.concentration-T0))*(0, -9.81))

navstokes.SolveInitial()

ld: warning: -undefined dynamic_lookup may not work with chained fixups
ld: warning: -undefined dynamic_lookup may not work with chained fixups


In [ ]:
scenev = Draw (navstokes.velocity, mesh, "velocity", vectors = { "grid_size" : 40 })
scenet = Draw (convdiff.concentration, mesh, "temp")

t, tend = 0, 1000
cnt = 0
with TaskManager():
    while t < tend:
        navstokes.DoTimeStep()
        convdiff.DoTimeStep()
        t = t+timestep
        cnt = cnt+1
        if cnt % 10 == 0:           
            print ('t = ', t, end='\r')
            scenev.Redraw()
            scenet.Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…